# Process Observation Data Analysis
## The aim of this analysis is to experiment with an alternative approach to analyzing process data that could potentially be more time efficient.

The data structure for the data can be seen in the image below. The observations performed vary in the level of completness as some have been observed more than others, while some have not been observed at all. And for some the sample size is large enough and variance low enough to give confidence that an average is representative of the sample.

![ERD Image](../Data/ERD.jpg)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import display
import seaborn as sns

In [4]:
data_folder = '../Data'
full_path = os.path.join(data_folder, 'ProcessData.xlsx')
excel_file_obj = pd.ExcelFile(full_path)
print('The names of the tabs or sheets in the excel file are as follows: ' ,excel_file_obj.sheet_names)

The names of the tabs or sheets in the excel file are as follows:  ['Iteration 1 Data (2)', 'Iteration 1 Data', 'Iteration 2 Tasks', 'Iteration 2 Session', 'DurationData', 'TaskData', 'RecordingData', 'ProcessData', 'HourlyRates']


From the list of sheets above it is clear that there are some additional unwanted sheets. Upon investigation it looks like the other sheets are templates that were imporoperly deleted and out of best practice we will not be dedicating resources to parsing them.

In [8]:
sheets_to_import = ['DurationData', 'TaskData', 'RecordingData', 'ProcessData', 'HourlyRates']
select_sheets = pd.read_excel(full_path, sheet_name=sheets_to_import)

In [10]:
display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration
15,16,11,5121200.0,00:14:03,00:14:26,0 days 00:00:23
138,139,40,511400.0,00:20:14,0:35:41,0 days 00:15:27
74,75,91,5122500.0,00:15:38,00:16:12,0 days 00:00:34
269,270,81,5322300.0,00:20:16,00:40:26,0 days 00:20:10
251,252,71,522300.0,01:47:15,01:52:28,0 days 00:05:13
195,196,90,5122200.0,00:36:38,00:36:44,0 days 00:00:06
29,30,20,5122400.0,01:23:22,01:28:48,0 days 00:05:26
451,453,130,413200.0,01:34:25,01:35:27,0 days 00:01:02
262,263,81,532100.0,00:00:17,00:00:44,0 days 00:00:27
111,112,61,5121200.0,03:02:50,03:02:57,0 days 00:00:07


For the DurationData table:
1. The TaskID needs to be fixed and the .0 needs to be removed
2. The Duration needs to be fixed, and the days need to be removed, or recalculate the Duration

In [13]:
select_sheets['DurationData']['TaskID'] = select_sheets['DurationData']['TaskID'].fillna(0).replace([float('inf'), float('-inf')], 0).astype(int)
display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration
468,470,130,413200,02:09:27,02:10:29,0 days 00:01:02
523,525,30,53841700,02:53:23,02:59:55,0 days 00:06:32
283,284,80,5122500,00:36:30,00:37:08,0 days 00:00:38
341,342,180,538411200,00:49:34,00:51:32,0 days 00:01:58
422,424,130,413100,00:33:08,00:37:16,0 days 00:04:08
27,28,20,5122200,01:23:00,01:23:17,0 days 00:00:17
435,437,130,413200,00:59:34,01:00:37,0 days 00:01:03
28,29,20,5122300,01:23:21,01:23:21,0 days 00:00:00
158,159,71,5122500,01:23:01,01:24:12,0 days 00:01:11
116,117,61,5121500,03:40:19,03:41:37,0 days 00:01:18


In [15]:
from datetime import datetime

In [19]:
select_sheets['DurationData']['StartTime'] = pd.to_datetime(select_sheets['DurationData']['StartTime'], format = '%H:%M:%S')
select_sheets['DurationData']['EndTime'] = pd.to_datetime(select_sheets['DurationData']['EndTime'], format = '%H:%M:%S')

select_sheets['DurationData'].loc[:,'Duration 2'] = select_sheets['DurationData']['EndTime'] - select_sheets['DurationData']['StartTime']

display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration,Duration 2
394,395,210,4313500,1900-01-01 00:09:56,1900-01-01 00:10:13,0 days 00:00:17,0 days 00:00:17
507,509,30,5371200,1900-01-01 01:25:22,1900-01-01 01:26:45,0 days 00:01:23,0 days 00:01:23
523,525,30,53841700,1900-01-01 02:53:23,1900-01-01 02:59:55,0 days 00:06:32,0 days 00:06:32
305,306,80,5322300,1900-01-01 02:00:35,1900-01-01 02:31:05,0 days 00:30:30,0 days 00:30:30
520,522,30,5371600,1900-01-01 01:46:36,1900-01-01 01:46:42,0 days 00:00:06,0 days 00:00:06
447,449,130,413100,1900-01-01 01:25:42,1900-01-01 01:27:36,0 days 00:01:54,0 days 00:01:54
348,349,180,5385500,1900-01-01 01:25:04,1900-01-01 01:40:14,0 days 00:15:10,0 days 00:15:10
156,157,71,5122400,1900-01-01 01:17:35,1900-01-01 01:20:06,0 days 00:02:31,0 days 00:02:31
519,521,30,5122500,1900-01-01 01:45:20,1900-01-01 01:46:34,0 days 00:01:14,0 days 00:01:14
91,92,61,5122100,1900-01-01 01:00:31,1900-01-01 01:01:16,0 days 00:00:45,0 days 00:00:45


In [21]:
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

select_sheets['DurationData'].loc[:,'Duration 3'] = select_sheets['DurationData']['Duration 2'].apply(format_timedelta)

display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration,Duration 2,Duration 3
30,31,20,5123300,1900-01-01 01:30:07,1900-01-01 01:30:07,0 days 00:00:00,0 days 00:00:00,00:00:00
411,412,130,413100,1900-01-01 00:02:00,1900-01-01 00:04:18,0 days 00:02:18,0 days 00:02:18,00:02:18
287,288,80,5122500,1900-01-01 00:40:07,1900-01-01 00:40:31,0 days 00:00:24,0 days 00:00:24,00:00:24
352,353,140,5371300,1900-01-01 00:03:53,1900-01-01 00:04:34,0 days 00:00:41,0 days 00:00:41,00:00:41
438,440,130,413200,1900-01-01 01:05:11,1900-01-01 01:06:47,0 days 00:01:36,0 days 00:01:36,00:01:36
425,427,130,413100,1900-01-01 00:42:16,1900-01-01 00:42:53,0 days 00:00:37,0 days 00:00:37,00:00:37
331,332,80,5223800,1900-01-01 03:38:56,1900-01-01 03:39:26,0 days 00:00:30,0 days 00:00:30,00:00:30
275,276,80,521600,1900-01-01 00:14:18,1900-01-01 00:14:29,0 days 00:00:11,0 days 00:00:11,00:00:11
393,394,210,4313100,1900-01-01 00:08:38,1900-01-01 00:09:52,0 days 00:01:14,0 days 00:01:14,00:01:14
336,337,180,5382100,1900-01-01 00:02:53,1900-01-01 00:06:57,0 days 00:04:04,0 days 00:04:04,00:04:04


To get the Duration in the right format we had to:
1. Convert the start and end time to datetime (datetime.datetime)
2. Calculate the difference (timedelta object)
3. Convert the start and end time to datetime.time to remove the year
4. Format the timedelta to remove the day count

In [24]:
select_sheets['DurationData']['StartTime'] = select_sheets['DurationData']['StartTime'].dt.time
select_sheets['DurationData']['EndTime'] = select_sheets['DurationData']['EndTime'].dt.time
display(select_sheets['DurationData'].sample(n=10))

,DurationID,RecordingID,TaskID,StartTime,EndTime,Duration,Duration 2,Duration 3
493,495,30,5372300,00:36:54,00:43:52,0 days 00:06:58,0 days 00:06:58,00:06:58
130,131,51,5122500,01:28:01,01:30:40,0 days 00:02:39,0 days 00:02:39,00:02:39
417,419,130,413400,00:15:01,00:19:52,0 days 00:04:51,0 days 00:04:51,00:04:51
321,322,80,5122500,03:20:46,03:21:01,0 days 00:00:15,0 days 00:00:15,00:00:15
273,274,80,5322200,00:10:28,00:16:55,0 days 00:06:27,0 days 00:06:27,00:06:27
342,343,180,539100,00:53:40,00:54:15,0 days 00:00:35,0 days 00:00:35,00:00:35
172,173,71,5121500,01:58:01,01:58:16,0 days 00:00:15,0 days 00:00:15,00:00:15
48,49,20,5121100,02:33:42,02:34:00,0 days 00:00:18,0 days 00:00:18,00:00:18
520,522,30,5371600,01:46:36,01:46:42,0 days 00:00:06,0 days 00:00:06,00:00:06
457,459,130,413100,01:52:05,01:54:18,0 days 00:02:13,0 days 00:02:13,00:02:13


In [14]:
display(select_sheets['TaskData'].sample(n=10))

,TaskID,TaskName,ProcessID,ResponsibleRole,Iterations,AverageTaskTime
130,5216500,Check service providers & add practice discipl...,521,Case manager,0,NaN
151,52231300,Check RPL codes on Medcode link,522,Pre-authorization Agent,0,NaN
118,52131100,Generate pre-authorisation confirmation & auth...,521,Pre-authorization Agent,0,NaN
170,4311100,Generate prepay run report,4311,BSS Technical Specialist,0,NaN
76,415500,Process claim,415,Claims Assessor,0,NaN
90,45100,Email reports,45,BI Team,0,NaN
73,415200,Capture claim,415,Claims Assessor,0,NaN
50,539100,Review case,539,Case manager,0,NaN
39,411300,Click Assess,411,Claims Assessor,0,NaN
183,5382200,Assess case for clinical appropriateness for e...,5382,Case manager,3,00:00:00


For the TaskData table:
1. We need to know up front how many processes we actually observed, then remove all processes that were not observed at least once
2. We need to know the number of iterations observed for each task, as well as how many people were observed. (Group and individualized views)
3. We need all measures of central tendency and either a box plot or scatter plot to get an idea of how times are distributed for each TaskID.
4. Find a relationship between number of iterations or people observed, and how data is distributed (looking at the mean as a proportion of the standard deviation)

In [20]:
display(select_sheets['RecordingData'])

,RecordingID,Recording Length,RecordingDate,PersonObserved,RecordingStatus
0,10.0,00:46:42,28/08/2024,Pauline Jordaan,Processed and Reviewed
1,11.0,00:29:04,28/08/2024,Pauline Jordaan,Processed and Reviewed
2,20.0,03:21:07,28/08/2024,Siyethemba Mkhize,Processed and Reviewed
3,30.0,03:03:36,29/08/2024,Jaqueline Wessels,Processed and Reviewed
4,40.0,00:42:28,02/09/2024,Zoleka Baba,Processed and Reviewed
...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN


For the RecordingData table:
1. The data type for Recording ID needs to be fixed
2. The extra rows need to be removed